In [ ]:
!pip install gym[atari]
!pip install ray==1.2 matplotlib
!pip install dm-tree dataclasses ray[rllib]

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import pickle
import requests
import gym
from ray.rllib.env.wrappers.atari_wrappers import wrap_deepmind
import matplotlib.pyplot as plt
from IPython import display


APP_URL = ''        # 部署服务的访问地址
D_NAME = ''              # 账号名
IAM_NAME = ''    # IAM用户名
PASSWORD = ''        # 密码


def create_env(name='PongNoFrameskip-v4'):
    env = gym.make(name)
    env = wrap_deepmind(
        env,
        dim=42,
        framestack=False,
        framestack_via_traj_view_api=False)
    return env


def prepare_test_config(name=IAM_NAME, domain_name=D_NAME, passwd=PASSWORD):
    config = {"app_url": APP_URL, "header_content_type": "text/plain", "local": False}
    header = {
        "Content-Type": "application/json"
    }
    body = {
        "auth": {
            "identity": {
                "methods": ["password"],
                "password": {
                    "user": {
                        "name": name,
                        "password": passwd,
                        "domain": {
                            "name": domain_name
                        }
                    }
                }
            },
            "scope": {
                "project": {
                    "name": "cn-north-4"
                },
            }
        }
    }
    body = json.dumps(body)
    response = requests.post("https://iam.cn-north-4.myhuaweicloud.com/v3/auth/tokens?nocatalog=true",
                             headers=header, data=body, verify=False)
    # print(response.headers['X-Subject-Token'])
    config["header_X_Auth_Token"] = response.headers['X-Subject-Token']
    print("config prepared")
    return config


class PolicyClient(object):
    """
    REST client to interact with a RLlib policy server.
    input param:
        configs (requires following dict):
            app_url
            local: True or False
            header_X_Auth_Token
            header_content_type

    """

    def __init__(self, configs):
        """
        configs must have following param:
            app_url
            valid_action
        """
        self.configs = configs
        self._address = self.configs["app_url"]
        self.env = create_env(name='PongNoFrameskip-v4')
        self.data = None
        self.parsed = None
        logger.info("address: {}".format(self._address))

    def _send(self, data):
        payload = pickle.dumps(data)
        header = {
            "X-Auth-Token": self.configs["header_X_Auth_Token"],
            "Content-Type": self.configs["header_content_type"]
        }
        response = requests.post(self._address, headers=header, data=payload, verify=False)
        if response.status_code != 200:
            logger.error("Request failed {}: {}".format(response.text, data))
        response.raise_for_status()
        self.parsed = pickle.loads(response.content)
        return self.parsed

    def launch(self):
        obs = self.env.reset()
        plt.imshow(obs)
        rewards = 0

        while True:
            resp = self._send({"obs": list(obs)})
            action = resp["action"]
            obs, reward, done, info = self.env.step(action)
            plt.imshow(obs)
            display.clear_output(wait=True)
            plt.pause(0.0000001)
            rewards += reward
            # print("log returns: ", reward)
            if done:
                print('Total rewards:', rewards)
                break
        plt.show()


config = prepare_test_config()
client = PolicyClient(config)
client.launch()